In [15]:
from bs4 import BeautifulSoup
import json
import requests
from time import sleep

In [3]:
f = open('ratebeer_brewery_hrefs.json','r')
hrefs = json.loads(f.read())

Get the page for each brewery that lists all the beers that they produce

In [55]:
brewery_pages = []
for h in hrefs:
    sleep(.1)
    url = 'https://www.ratebeer.com' + h
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text,'lxml')
    try:
        if soup.find('div', {'id': 'brewer-table'}).find('table', {'id':'brewer-beer-table'}): # some pages have the
            brewery_pages.append(h)                                                    # table of beers on this page
    except:
        brewery_pages.append(soup.find('div', {'id': 'brewer-table'}).find('a')['href']) # some link to the page

This next bit doesn't get all the links because page only loads on scroll, had to use selenium in another script. Stored the results from that script in beer_pages.json

In [61]:
beer_pages = []
for p in brewery_pages:
    url = 'https://www.ratebeer.com' + p
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text,'lxml')
    for tr in soup.find('table', {'id': 'brewer-beer-table'}).find('tbody').findAll('tr'):
        beer_pages.append(tr.find('td').find('a')['href'])

Exporting the brewery pages in order to run the above code with selenium (on my local machine -- this was written on a remote machine)

In [72]:
j = json.dumps(brewery_pages)
f = open('brewery_pages.json','w')
f.write(j)
f.close()

In [74]:
len([c for c in beer_pages if 'mikkeller' in c]) # this should be 919

83

Will scrape each beer page, along with reviews and put into mongodb in another script (ratebeer_mongo.ipynb)